In [ ]:
#SECTION 1 - IMPORTING PACKAGES

import PySimpleGUI as sg
import autokeras as ak
import calendar
import datetime
import locale
import nlp_id
import ntpath
import numpy as np 
import os
import pandas as pd
import pickle
import plotly.express as px
import random
import re
import shutil
import sklearn
import tensorflow
import webbrowser
from nlp_id.lemmatizer import Lemmatizer
from nlp_id.postag import PosTag
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import load_model

#SECTION 2 - DECLARATIONS

##Here I set the display settings of pandas' DataFrames. This is required so that in the finder function, the multiline will display the search result without truncation.
pd.set_option('expand_frame_repr', False)
pd.set_option('display.max_colwidth', None)
pd.set_option('mode.chained_assignment', None)
##Here I load the location database.
location_data=pd.read_csv(r"CONST/location_data.csv", sep=None, decimal=",", index_col=False, engine='python', dtype={'Branch Code': str, 'KCU Code': str})
location_data['Latitude'] = pd.to_numeric(location_data['Latitude'])
location_data['Longitude'] = pd.to_numeric(location_data['Longitude'])
##Here I declare settings to display month names.
locale.setlocale(locale.LC_TIME, "id_ID")
mos = calendar.month_name[1:]
##Here I set up the POS tagger filter. Strictly do not modify the line directly below.
pos_db = ["FW", "JJ", "NN", "VB", "NNP", "IN", "NEG"]
##Here I load SKA & Divisi constants and map colors.
with open("CONST/idxs.txt") as f: 
    idxs = eval(f.read())
with open("CONST/colors.txt") as f: 
    cdm = eval(f.read())
##Here I declare question numbers based on their category.
bsq_l=[4,6,11]
bsq_d=[8,13,15]
bsq=bsq_l+bsq_d+[19,22]
ce_be=[5]
ce_br=[7]
ce=ce_be+ce_br+[14]
all=['all']
##Here I load built-in nlp_id functions.
lemmatizer = Lemmatizer() 
postagger = PosTag()

#SECTION 3 - METHODS

##Here I write a custom filter to POS tag and lemmatize sentences.
def process(data):
    for x in range(len(data)):
        l=""
        m=data[x]
        u=pd.DataFrame(postagger.get_pos_tag(m), columns=['word','pos'])
        for y in range(len(u)):
            if u.at[y,'pos'] in pos_db:
                l = " ".join([l,u.at[y,'word']])
        l = lemmatizer.lemmatize(l)
        data[x] = l
    return (data)

##Here, modified from https://www.tutorialspoint.com/find-the-most-similar-sentence-in-the-file-to-the-input-sentence-nlp,
##   I build a Tf-idf vectorizer that uses my custom sentence processor, sorts based on score, and takes top n results.
def get_most_similar_sentences(sta, data, novs):
    temp = data.loc[:, 'Verbatim']
    sentences = list(lemmatizer.lemmatize(temp[x]) for x in range(len(data)))
    sta_p = lemmatizer.lemmatize(sta)

    #these 3 lines are unchanged
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform([sta_p] + sentences)
    similarity_scores = (tfidf_matrix * tfidf_matrix.T).A[0][1:]

    data['sim'] = similarity_scores
    novs = novs if novs < len(sentences) else len(sentences)
    most_similar_indices = np.argpartition(similarity_scores, -novs)[-novs:]
    most_similar_sentences = data.loc[data.index.isin(most_similar_indices)].copy()
    most_similar_sentences.sort_values(by='sim', ascending=False, inplace=True)
    most_similar_sentences.drop('sim', axis=1, inplace=True)
    return most_similar_sentences

#SECTION 4 - GUI SETTINGS

##Font Styles and Size
tmf=("Arial", 20)
bmf=("Arial", 15)

##Buttons
###Main Menu
button_model=sg.Button("Categorize Data", key='model', font=bmf)
button_map=sg.Button("View Map", key='map', font=bmf)
button_finder=sg.Button("Verbatim Finder", key='finder', font=bmf)
###Model
button_cat=sg.Button("Kategorisasi data berdasarkan verbatim", key='cat')
button_train=sg.Button("Gunakan data untuk melatih model", key='tra', disabled=True)  #Temporarily disabled until autokeras fixes the KeyError: 'text_block_1/max_tokens' error
button_backm=sg.Button("Kembali ke menu", key='backm')
###Map
button_update=sg.Button("Generate map", key='upd')
button_backs=sg.Button("Kembali ke menu", key='backs')
###Finder
button_selectall=sg.Button("Pilih semua", key='sall')
button_unselectall=sg.Button("Kosongkan semua", key='uall')
button_run=sg.Button("Run", key='run')
button_backf=sg.Button("Kembali ke menu", key='backf')

##Checkboxes
###Model
check_csv=sg.Check('CSV', key='c_csv')
check_excel=sg.Check('Excel', key='c_xls')
###Map
check_kws=sg.Check('Kanwil', key='c_k', enable_events=True, disabled=True)
check_years=sg.Check('Kurun waktu', key='c_y', enable_events=True, disabled=True)
check_vbs=sg.Check('Jenis verbatim', key='c_v', enable_events=True, disabled=True)
###Finder
check_kanwils=[[sg.Check('1', key='1'), sg.Check('2', key='2'), sg.Check('3', key='3'), sg.Check('4', key='4'), sg.Check('5', key='5'), sg.Check('6', key='6')],[sg.Check('7', key='7'), sg.Check('8', key='8'), sg.Check('9', key='9'), sg.Check('10', key='10'), sg.Check('11', key='11'), sg.Check('12', key='12')]]
check_savef=sg.Check('Simpan hasil pencarian', key='savef')

##Combo Boxes
###Map
combo_kws=sg.Combo([i for i in range(1, 13)], key='kws', disabled=True, readonly=True, enable_events=True)
combo_kcu=sg.Combo([], key='kcu', disabled=True, size=(30,1), readonly=True)
combo_mons=sg.Combo(mos, key='months_start', disabled=True, readonly=True, default_value='')
combo_mone=sg.Combo(mos, key='months_end', disabled=True, readonly=True, default_value='')
combo_qts=sg.Combo(["BSQ", "CE", "Semua"], key='qts', disabled=True, readonly=True, default_value="Semua")
###Finder
combo_qtf=sg.Combo(["Semua Verbatim", "BSQ Layanan", "BSQ Digital", "CE Branch Experience", "CE Branch Relationship"], default_value="Semua Verbatim" ,key='qtf', readonly=True)

##File Browser
file_browserm=sg.FileBrowse(key='filebrowsem', file_types=(("","*.xls *.xlsx *.csv"),)) #Model - suffix M
file_browsers=sg.FileBrowse(key='filebrowses', file_types=(("","*.xls *.xlsx *.csv"),)) #Map - suffix S
file_browserf=sg.FileBrowse(key='filebrowsef', file_types=(("","*.xls *.xlsx *.csv"),)) #Finder - suffix F

##Input Fields
###File Input
input_filem=sg.Input("", key='filem', disabled=True, enable_events=True) #Model
input_files=sg.Input("", key='files', disabled=True, enable_events=True) #Map
input_filef=sg.Input("", key='filef', disabled=True, enable_events=True) #Finder
###Input Sentence (Finder)
input_sentence=sg.Input("", key='sentence')

##Multiline Displays
multi=sg.Multiline(key='results', visible=False, size=(150, 10)) #Finder

##Spins
###Map
spin_yrss=sg.Spin([], initial_value='', key='yrs_start', disabled=True, readonly=True)
spin_yrse=sg.Spin([], initial_value='', key='yrs_end', disabled=True, readonly=True)
###Finder
spin_amt=sg.Spin([1], initial_value=1, key='novs', readonly=True)

##Static Text
header=sg.Text("Verbatim Analyzer", key='header', font=tmf)

##Layouts
###Main Menu Wrap
all_els=[[button_model,button_map,button_finder]]
###Main Menu Layout
main_layout = [[sg.Column(all_els,element_justification='c')]]
###Model Menu Layout
model_layout = [
    [sg.Text("Pilih file"), input_filem, file_browserm],
    [button_cat, sg.Text("Simpan sebagai"), check_csv, check_excel],
    [sg.Text("atau"), button_train, sg.Push(), sg.Push(), button_backm]
]
###Map Menu Layout
map_layout = [
    [sg.Text("Pilih file"), input_files, file_browsers],
    [sg.Text("Filter dan kelompokkan data berdasarkan:")],
    [check_kws, combo_kws, sg.Text("KCU"), combo_kcu],
    [check_years, sg.Text("Dari"), combo_mons, spin_yrss, sg.Text("Sampai"), combo_mone, spin_yrse],
    [check_vbs, combo_qts],
    [button_update, sg.Push(), sg.Push(), button_backs]
]
###Finder Menu Layout
finder_layout = [
    [sg.Text("Pilih file"), input_filef, file_browserf],
    [sg.HorizontalSeparator()],
    [sg.Text("Kriteria teks:"), input_sentence],
    [sg.HorizontalSeparator()],
    [sg.Text("Kanwil area pencarian:")],
    [sg.Column(check_kanwils)],
    [button_selectall, button_unselectall],
    [sg.HorizontalSeparator()],
    [sg.Text("Cari "), spin_amt, sg.Text(" verbatim paling relevan")],
    [sg.HorizontalSeparator()],
    [sg.Text("Question type:"), combo_qtf],
    [sg.HorizontalSeparator()],
    [button_run, check_savef, sg.Push(), sg.Push(), button_backf],
    [multi],
]
###Main Layout
layout = [[header],[sg.Column(main_layout, key='mainmenu', visible=True), sg.Column(model_layout, visible=False, key='modelmenu'), sg.Column(map_layout, visible=False, key='mapmenu'), sg.Column(finder_layout, visible=False, key='findermenu')]]
##Window
window = sg.Window("Verbatim Analyzer", layout)

#SECTION 5 - MAIN PROGRAM

while True:
    event, values = window.Read()
    if event in (None, 'Exit'):
        break
##MAIN MENU FUNCTIONALITIES
###Load and Initialize Data Categorizer
    if event == "model":
        window['header'].update("Verbatim Categorizer")
        window['mainmenu'].update(visible=False)
        window['modelmenu'].update(visible=True)
        window['filem'].update(value='')
        window['c_csv'].update(False)
        window['c_xls'].update(False)
        fta=""
        data = pd.DataFrame()
###Load and Initialize Map Generator
    if event == "map":
        window['header'].update("Map Generator")
        window['mainmenu'].update(visible=False)
        window['mapmenu'].update(visible=True)
        window['files'].update(value='')
        window['c_k'].update(False, disabled=True)
        window['c_v'].update(False, disabled=True)
        window['c_y'].update(False, disabled=True)
        window['kws'].update(value='',disabled=True)
        window['kcu'].update(value='',disabled=True)
        window['months_start'].update(value='',disabled=True)
        window['months_end'].update(value='',disabled=True)
        window['yrs_start'].update(value='',disabled=True)
        window['yrs_end'].update(value='',disabled=True)
        window['qts'].update(value='Semua',disabled=True)
        fta=""
        data = pd.DataFrame()
###Load and Initialize Verbatim Finder
    if event == "finder":
        window['header'].update("Verbatim Finder")
        window['mainmenu'].update(visible=False)
        window['findermenu'].update(visible=True)
        window['filef'].update(value='')
        window['sentence'].update(value='')
        for x in range (1, 13):
            window[str(x)].update(False)
        window['novs'].update(value=1)
        window['qtf'].update(value='Semua Verbatim')
        window['savef'].update(False)
        window['results'].update(value='', visible=False)
        fta=""
        data = pd.DataFrame()
##DATA CATEGORIZER FUNCTIONALITIES
###File Reader
    if event == "filem":
        fta=values['filem']
###Categorize Data
    if event == "cat":
        if fta.lower().endswith('.csv'):    
            try:
                data = pd.read_csv(fta, engine='python', sep=None)
                data.columns= data.columns.str.title()
                vl = [col for col in data.columns if "Verbatim" in col][0]
                data.rename(columns = {vl:'Verbatim'}, inplace = True)
                X=data["Verbatim"].values.tolist()
                gate=True
            except:
                sg.Popup('Data tidak valid!')
                data = pd.DataFrame()
                fta=""
                gate=False
            if gate:
                X=process(X)
                X=np.asarray(X)
                np.object = object
                np.unicode = str
                shutil.copytree('TC_SKA', os.getcwd(), dirs_exist_ok = True)
                m=load_model("model_ska")
                m=ak.TextClassifier(m)
                data['Ska'] = m.predict(X, verbose=0)
                for L in range(len(data)):
                    data.at[L,'Ska'] = idxs['Ska'][int(data.at[L,'Ska'])]
                shutil.copytree('TC_DIV', os.getcwd(), dirs_exist_ok = True)
                n=load_model("model_div")
                n=ak.TextClassifier(n)
                data['Divisi'] = n.predict(X, verbose=0)
                data['Sub Divisi'] = 'x'
                for L in range(len(data)):
                    data.at[L,'Divisi'] = idxs['Divisi'][int(data.at[L,'Divisi'])]
                    with open("CONST/subdiv.txt") as f: 
                        subdiv_search = eval(f.read())
                    with open("CONST/fb.txt") as f: 
                        fb = eval(f.read())
                    try:
                        qn = data.at[L,'Question Type']
                    except:
                        qn = None
                    if fb.get(qn) is not None and data.at[L,'Divisi'] == 'FB': 
                        data.at[L,'Sub Divisi'] = fb[qn]
                    elif data.at[L,'Divisi'] in subdiv_search:
                        search_list=list(subdiv_search[data.at[L,'Divisi']].keys())
                        for Q in range(len(search_list)):
                            if search_list[Q] in re.sub('\W+','',data.at[L,"Verbatim"].lower()):
                                data.at[L,'Sub Divisi'] = subdiv_search[data.at[L,'Divisi']][search_list[Q]]
                fn=ntpath.basename(fta)
                if values['c_csv'] == True:
                    data.to_csv(fn+" - Categorized.csv", sep=';', index=False)
                if values['c_xls'] == True:
                    data.to_excel(fn+" - Categorized.xlsx", index=False)
                try:
                    shutil.rmtree("model_ska")
                    shutil.rmtree("model_div")
                    shutil.rmtree("text_classifier")
                except:
                    pass
                sg.popup("Data sudah selesai diproses.")    
        else:
            ds=pd.ExcelFile(fta)
            big_gate=False
            for sn in range (len(ds.sheet_names)):
                try:
                    data=pd.read_excel(fta, sheet_name=ds.sheet_names[sn], engine='openpyxl')
                    data.columns= data.columns.str.title()
                    vl = [col for col in data.columns if "Verbatim" in col][0]
                    data.rename(columns = {vl:'Verbatim'}, inplace = True)
                    X=data["Verbatim"].values.tolist()
                    gate=True
                    big_gate=True
                except Exception as e:
                    sg.Popup('Sheet '+ds.sheet_names[sn]+' tidak valid: '+str(e))
                    data = pd.DataFrame()
                    gate=False
                if gate:
                    X=process(X)
                    X=np.asarray(X)
                    np.object = object
                    np.unicode = str
                    shutil.copytree('TC_SKA', os.getcwd(), dirs_exist_ok = True)
                    m=load_model("model_ska")
                    m=ak.TextClassifier(m)
                    data['Ska'] = m.predict(X, verbose=0)
                    for L in range(len(data)):
                        data.at[L,'Ska'] = idxs['Ska'][int(data.at[L,'Ska'])]
                    shutil.copytree('TC_DIV', os.getcwd(), dirs_exist_ok = True)
                    n=load_model("model_div")
                    n=ak.TextClassifier(n)
                    data['Divisi'] = n.predict(X, verbose=0)
                    data['Sub Divisi'] = 'x'
                    for L in range(len(data)):
                        data.at[L,'Divisi'] = idxs['Divisi'][int(data.at[L,'Divisi'])]
                        with open("CONST/subdiv.txt") as f: 
                            subdiv_search = eval(f.read())
                        with open("CONST/fb.txt") as f: 
                            fb = eval(f.read())
                        try:
                            qn = data.at[L,'Question Type']
                        except:
                            qn = None
                        if fb.get(qn) is not None and data.at[L,'Divisi'] == 'FB':
                            data.at[L,'Sub Divisi'] = fb[qn]
                        elif data.at[L,'Divisi'] in subdiv_search:
                            search_list=list(subdiv_search[data.at[L,'Divisi']].keys())
                            for Q in range(len(search_list)):
                                if search_list[Q] in re.sub('\W+','',data.at[L,"Verbatim"].lower()):
                                    data.at[L,'Sub Divisi'] = subdiv_search[data.at[L,'Divisi']][search_list[Q]]
                    fn=ntpath.basename(fta)
                    if values['c_csv'] == True:
                        data.to_csv(fn+" "+ds.sheet_names[sn]+" - Categorized.csv", sep=';', index=False)
                    if values['c_xls'] == True:
                        try:
                            with pd.ExcelWriter(fn+" - Categorized.xlsx", mode="a") as writer:
                                data.to_excel(writer, engine='xlsxwriter', index=False, sheet_name=ds.sheet_names[sn])
                        except:
                            with pd.ExcelWriter(fn+" - Categorized.xlsx", mode="w") as writer:
                                data.to_excel(writer, engine='xlsxwriter', index=False, sheet_name=ds.sheet_names[sn])
            if not big_gate:
                sg.Popup('Data tidak valid!')
                fta=""
            else:
                try:
                    shutil.rmtree("model_ska")
                    shutil.rmtree("model_div")
                    shutil.rmtree("text_classifier")
                except:
                    pass
                sg.popup("Data sudah selesai diproses.")
###Retrain Model - Currently Disabled Due to AutoKeras Bug
    if event == "tra":
        if fta.lower().endswith('.csv'): 
            try:
                data = pd.read_csv(fta, engine='python', sep=None)
                vl = [col for col in data.columns if "Verbatim" in col][0]
                data.rename(columns = {vl:'Verbatim'}, inplace = True)
                data = data[["Verbatim", "Ska", "Divisi"]]
            except:
                data = pd.DataFrame()
        else:
            data = pd.DataFrame()
            ds = pd.ExcelFile(fta)
            for sn in range (len(ds.sheet_names)):
                try:
                    dr=pd.read_excel(fta, sheet_name=ds.sheet_names[sn], engine='openpyxl')
                    dr.columns = dr.columns.str.title()
                    vl = [col for col in dr.columns if "Verbatim" in col][0]
                    dr.rename(columns = {vl:'Verbatim'}, inplace = True)
                    if not data.empty:
                        try:
                            data = pd.concat([data,dr[["Verbatim", "Ska", "Divisi"]]],ignore_index=True)
                        except Exception as e:
                            sg.Popup('Sheet '+ds.sheet_names[sn]+' tidak valid: '+str(e))
                    else:
                        try:
                            data = dr[["Verbatim", "Ska", "Divisi"]]
                        except Exception as e:
                            sg.Popup('Sheet '+ds.sheet_names[sn]+' tidak valid: '+str(e))
                except Exception as e:
                    sg.Popup('Sheet '+ds.sheet_names[sn]+' tidak valid: '+str(e))
        if data.empty:
            sg.Popup('Data tidak valid!')
            fta=""
        else:    
            X=data["Verbatim"]
            y=data["Ska"]
            w=data["Divisi"]
            X=X.tolist()
            y=y.tolist()
            w=w.tolist()
            for L in range(0, len(y)):
                try:
                    y[L] = idxs["Ska"].index(y[L])
                except:
                    idxs["Ska"].append(y[L])
                    y[L] = idxs["Ska"].index(y[L])
                    with open('CONST/idxs.txt','w') as data:  
                        data.write(str(idxs))
            for L in range(0, len(w)):
                try:
                    w[L] = idxs["Divisi"].index(w[L])
                except:
                    idxs["Divisi"].append(y[L])
                    w[L] = idxs["Divisi"].index(w[L])
                    with open('CONST/idxs.txt','w') as data:  
                        data.write(str(idxs))
            X=np.asarray(X)
            y=np.asarray(y)
            w=np.asarray(w)
            X=X.reshape(-1,1)
            y=y.reshape(-1,1)
            w=w.reshape(-1,1)
            np.object = object
            np.unicode = str
            shutil.copytree('TC_SKA', os.getcwd(), dirs_exist_ok = True)
            m=load_model("model_ska")
            m=ak.TextClassifier(m)
            m.fit(X, y, epochs=15, validation_split=0.1, verbose=0)
            try:
                m.save("TC_SKA/model_ska", save_format="tf")
            except Exception:
                m.save("TC_SKA/model_ska.h5")
            shutil.copytree('TC_DIV', os.getcwd(), dirs_exist_ok = True)
            n=load_model("model_div")
            n=ak.TextClassifier(n)
            n.fit(X, w, epochs=15, validation_split=0.1, verbose=0)
            try:
                m.save("TC_DIV/model_div", save_format="tf")
            except Exception:
                m.save("TC_DIV/model_div.h5")
            shutil.rmtree("model_ska")
            shutil.rmtree("model_div")
            shutil.rmtree("text_classifier")
            sg.popup("Data sudah selesai diproses.")
    if event == "backm": #back to menu from model
        window['header'].update("Verbatim Analyzer")
        window['mainmenu'].update(visible=True)
        window['modelmenu'].update(visible=False)
##MAP GENERATOR FUNCTIONALITIES
###File Reader
    if event == "files":
        data = pd.DataFrame()
        fta=values['files']
        if fta.lower().endswith('.csv'):
            try:
                dr = pd.read_csv(fta, engine='python', sep=None, dtype={'Branch Code': str}, parse_dates=["Periode"], dayfirst=True)
                dr.columns = dr.columns.str.title()
                vl = [col for col in dr.columns if "Verbatim" in col][0]
                dr.rename(columns = {vl:'Verbatim'}, inplace = True)
                kl = [col for col in dr.columns if "Kanwil" in col][0]
                dr.rename(columns = {kl:'Kanwil'}, inplace = True)
                data = dr[["Periode", "Kanwil", "Branch Code", "Question Type", "Verbatim", "Ska"]]
                gate = True
            except Exception as e:
                sg.Popup('Data tidak valid: '+str(e))
                data = pd.DataFrame()
                fta=""
                gate = False
        else:
            ds = pd.ExcelFile(fta)
            for sn in range (len(ds.sheet_names)):
                try:
                    dr=pd.read_excel(fta, sheet_name=ds.sheet_names[sn], engine='openpyxl', dtype={'Branch Code': str}, parse_dates=["Periode"])
                    dr.columns = dr.columns.str.title()
                    vl = [col for col in dr.columns if "Verbatim" in col][0]
                    dr.rename(columns = {vl:'Verbatim'}, inplace = True)
                    kl = [col for col in dr.columns if "Kanwil" in col][0]
                    dr.rename(columns = {kl:'Kanwil'}, inplace = True)
                    if not data.empty:
                        try:
                            data = pd.concat([data,dr[["Periode", "Kanwil", "Branch Code", "Question Type", "Verbatim", "Ska"]]],ignore_index=True)
                        except Exception as e:
                            sg.Popup('Sheet '+ds.sheet_names[sn]+' tidak valid: '+str(e))
                    else:
                        try:
                            data = dr[["Periode", "Kanwil", "Branch Code", "Question Type", "Verbatim", "Ska"]]
                        except Exception as e:
                            sg.Popup('Sheet '+ds.sheet_names[sn]+' tidak valid: '+str(e))
                except Exception as e:
                    sg.Popup('Sheet '+ds.sheet_names[sn]+' tidak valid: '+str(e))
            if not data.empty:
                gate = True
            else:
                sg.Popup('Data tidak valid!')
                fta=""
                gate = False
        if gate:
            data=data.replace('x', '')
            data["year"]=data["Periode"].dt.year
            for L in range(len(data)):
                data.at[L,"month"]=calendar.month_name[data.at[L,"Periode"].month]
            window['c_k'].update(disabled=False)
            window['c_y'].update(disabled=False)
            window['c_v'].update(disabled=False)
        else:
            window['kws'].update(value='',disabled=True)
            window['kcu'].update(value='',disabled=True)
            window['months_start'].update(value='',disabled=True)
            window['months_end'].update(value='',disabled=True)
            window['yrs_start'].update(value='',disabled=True)
            window['yrs_end'].update(value='',disabled=True)
            window['qts'].update(value='Semua',disabled=True)
            window['c_k'].update(False, disabled=True)
            window['c_y'].update(False, disabled=True)
            window['c_v'].update(False, disabled=True)
###Select Kanwil/KCU
    if event == "c_k":
        if values['c_k'] == True: #if enabled, set data
            window['kws'].update(value=1, disabled=False)
            kk=location_data[(location_data["Branch Code"] == location_data["KCU Code"]) & (location_data["Kanwil"] == 1)]["Branch Name"].values.tolist()
            window['kcu'].update(values=kk, disabled=False)
        else: #if disabled, clear
            window['kws'].update(value='', disabled=True)
            window['kcu'].update(value='', disabled=True)
            kk=[]
###If Kanwil Selection Changed, Update KCU List
    if event == "kws":
        kk=location_data[(location_data["Branch Code"] == location_data["KCU Code"]) & (location_data["Kanwil"] == int(values["kws"]))]["Branch Name"].values.tolist()
        window['kcu'].update(values=kk, disabled=False)
###Select Timeframe
    if event == "c_y":
        if values['c_y'] == True: #if enabled, set data
            window['months_start'].update(value=mos[min(data["Periode"]).month-1], disabled=False)
            window['months_end'].update(value=mos[max(data["Periode"]).month-1], disabled=False)
            window['yrs_start'].update(value=min(data["Periode"]).year, values=list(range((min(data["Periode"]).year), (max(data["Periode"]).year)+1)), disabled=False)
            window['yrs_end'].update(value=max(data["Periode"]).year, values=list(range((min(data["Periode"]).year), (max(data["Periode"]).year)+1)), disabled=False)
        else: #if disabled, clear
            window['months_start'].update(value='',disabled=True)
            window['months_end'].update(value='',disabled=True)
            window['yrs_start'].update(value='',disabled=True)
            window['yrs_end'].update(value='',disabled=True)
###Select Question Type
    if event == "c_v":
        if values['c_v'] == True:
            window['qts'].update(disabled=False)
        else:
            window['qts'].update(value='Semua', disabled=True)
###Generate Map
    if event == "upd":
        fn=ntpath.basename(fta)
        if data.empty: #error handling
            sg.Popup('Data tidak valid!')
        else:
            data_proj = data.drop(data.index[(data['Ska'] == '')])
            if values['c_y'] == True: #timeframe filter
                date_from = pd.Timestamp(datetime.date(values['yrs_start'], mos.index(values['months_start'])+1, 1))
                date_to = pd.Timestamp(datetime.date(values['yrs_end'], mos.index(values['months_end'])+1, 1))
                if date_from>date_to:
                    date_from = pd.Timestamp(datetime.date(values['yrs_start'], mos.index(values['months_start'])+1, calendar.monthrange(values['yrs_start'], mos.index(values['months_start'])+1)[1]))
                    kd = (data_proj["Periode"] >= date_to) & (data_proj["Periode"] <= date_from)
                    fn = fn + " Periode " + values['months_end'] + " " + str(values['yrs_end']) + " - " + values['months_start'] + " " + str(values['yrs_start'])
                else:
                    date_to = pd.Timestamp(datetime.date(values['yrs_end'], mos.index(values['months_end'])+1, calendar.monthrange(values['yrs_end'], mos.index(values['months_end'])+1)[1]))
                    kd = (data_proj["Periode"] >= date_from) & (data_proj["Periode"] <= date_to)
                    fn = fn + " Periode " + values['months_start'] + " " + str(values['yrs_start']) + " - " + values['months_end'] + " " + str(values['yrs_end'])
                data_proj=data_proj.loc[kd]
            if values['c_k'] == True: #kanwil filter
                if values['kcu'] != '':
                    kcuc=location_data[location_data["KCU Code"] == location_data.loc[location_data["Branch Name"]==values['kcu'],"Branch Code"].values[0]]["Branch Code"].values.tolist()
                    data_proj = data_proj.loc[data_proj['Branch Code'].isin(kcuc)]
                    fn = fn + " KCU " + values['kcu']
                elif values['kws'] != '':
                    data_proj = data_proj.loc[data_proj[[col for col in data.columns if "Kanwil" in col][0]]==int(values['kws'])]
                    fn = fn + " Kanwil " + str(values['kws'])
            if values['c_v'] == True: #question type filter
                if values['qts']=="BSQ":
                    data_proj['Question Type'] = data_proj['Question Type'].str.replace('\D', '', regex=True)
                    data_proj['Question Type'] = pd.to_numeric(data_proj['Question Type'])
                    data_proj = data_proj[data_proj['Question Type'].isin(bsq)]
                    fn = fn + " - BSQ"
                elif values['qts']=="CE":
                    data_proj['Question Type'] = data_proj['Question Type'].str.replace('\D', '', regex=True)
                    data_proj['Question Type'] = pd.to_numeric(data_proj['Question Type'])
                    data_proj = data_proj[data_proj['Question Type'].isin(ce)]
                    fn = fn + " - CE"
                else:
                    fn = fn + " - BSQ & CE"
            else:
                fn = fn + " - BSQ & CE"

            if not (data_proj.empty):
                location_data_proj = location_data.copy()
                location_data_proj = location_data_proj.assign(Count=location_data_proj['Branch Code'].map(data_proj['Branch Code'].value_counts()))
                location_data_proj["SKA_M"]=""
                location_data_proj["SKA_M_C"]=""
                location_data_proj["SKA_M_2"]=""
                location_data_proj["SKA_M_C_2"]=""
                location_data_proj.dropna(subset=['Count'], inplace=True)
                data_proj = data_proj.reset_index(drop=True)
                location_data_proj = location_data_proj.reset_index(drop=True)
                for L in range(len(location_data_proj)):
                    try:
                        location_data_proj.at[L,"SKA_M"] = data_proj.loc[data_proj["Branch Code"]==location_data_proj.at[L,'Branch Code'],'Ska'].mode()[0]
                        location_data_proj.at[L,"SKA_M_C"] = len(data_proj.loc[(data_proj["Branch Code"]==location_data_proj.at[L,'Branch Code']) & (data_proj["Ska"]==location_data_proj.at[L,"SKA_M"])])
                    except:
                        location_data_proj.at[L,"SKA_M"] = "Tidak Ada"
                        location_data_proj.at[L,"SKA_M_C"] = 0
                    try:
                        location_data_proj.at[L,"SKA_M_2"] = data_proj.loc[data_proj["Branch Code"]==location_data_proj.at[L,'Branch Code'],'Ska'].value_counts().index[1]
                        location_data_proj.at[L,"SKA_M_C_2"] = len(data_proj.loc[(data_proj["Branch Code"]==location_data_proj.at[L,'Branch Code']) & (data_proj["Ska"]==location_data_proj.at[L,"SKA_M_2"])])
                    except:
                        location_data_proj.at[L,"SKA_M_2"] = "Tidak Ada"
                        location_data_proj.at[L,"SKA_M_C_2"] = 0
                location_data_proj["SKA_M_C"] = location_data_proj["SKA_M_C"].astype(float)
                location_data_proj["SKA_M_C_2"] = location_data_proj["SKA_M_C_2"].astype(float)
                location_data_proj = location_data_proj.reset_index(drop=True)
                vis = px.scatter_mapbox(location_data_proj, lat="Latitude", lon="Longitude", size="SKA_M_C", title="Map - " + fn, color='SKA_M', mapbox_style="carto-positron", color_discrete_map=cdm, custom_data=['Count', 'SKA_M', 'SKA_M_C', 'SKA_M_2', 'SKA_M_C_2', 'Branch Name'])
                vis.update_traces(hovertemplate="<b>%{customdata[5]}</b><br><br><b>Verbatim Terbanyak: %{customdata[1]}</b><br>dengan jumlah <b>%{customdata[2]} dari total %{customdata[0]} verbatim.</b><br>Verbatim kedua terbanyak: %{customdata[3]} (%{customdata[4]})<extra></extra>")
                vis.update_layout(autosize = True, legend_title="SKA")
                vis.write_html(f"Map - {fn}.html")
                webbrowser.open_new_tab(f"Map - {fn}.html")
                sg.popup("Data sudah selesai diproses.")
            else:
                sg.Popup('Tidak ada data yang sesuai filter!')
    if event == "backs": #back to main menu from map
        window['header'].update("Verbatim Analyzer")
        window['mainmenu'].update(visible=True)
        window['mapmenu'].update(visible=False)
##FINDER FUNCTIONALITIES
###File Reader
    if event == "filef":
        data = pd.DataFrame()
        fta=values['filef']
        if fta.lower().endswith('.csv'):
            try:
                dr = pd.read_csv(fta, sep=None, engine='python')
                dr.columns = dr.columns.str.title()
                vl = [col for col in dr.columns if "Verbatim" in col][0]
                dr.rename(columns = {vl:'Verbatim'}, inplace = True)
                kl = [col for col in dr.columns if "Kanwil" in col][0]
                dr.rename(columns = {kl:'Kanwil'}, inplace = True)
                data = dr[["Kanwil", "Question Type", "Verbatim"]]
                window['novs'].update(values=[i for i in range(1, len(data))])  #update number of verbatims spin
            except Exception as e:
                sg.Popup('Data tidak valid: '+str(e))
                window['novs'].update(values=[1])
                fta=""
        else:
            ds = pd.ExcelFile(fta)
            for sn in range (len(ds.sheet_names)):
                try:
                    dr=pd.read_excel(fta, sheet_name=ds.sheet_names[sn], engine='openpyxl')
                    dr.columns = dr.columns.str.title() #rename columns
                    vl = [col for col in dr.columns if "Verbatim" in col][0]
                    dr.rename(columns = {vl:'Verbatim'}, inplace = True)
                    kl = [col for col in dr.columns if "Kanwil" in col][0]
                    dr.rename(columns = {kl:'Kanwil'}, inplace = True)
                    if not data.empty:
                        try:
                            data = pd.concat([data,dr[["Kanwil", "Question Type", "Verbatim"]]],ignore_index=True)
                        except Exception as e:
                            sg.Popup('Sheet '+ds.sheet_names[sn]+' tidak valid: '+str(e))
                    else:
                        try:
                            data = dr[["Kanwil", "Question Type", "Verbatim"]]
                        except Exception as e:
                            sg.Popup('Sheet '+ds.sheet_names[sn]+' tidak valid: '+str(e))
                except Exception as e:
                    sg.Popup('Sheet '+ds.sheet_names[sn]+' tidak valid: '+str(e))
            if data.empty:
                sg.Popup('Data tidak valid!')
                window['novs'].update(values=[1])
                fta=""
            else:
                window['novs'].update(values=[i for i in range(1, len(data))]) #update number of verbatims spin
###Select All Kanwils
    if event == "sall":
        for x in range (1, 13):
            window[str(x)].update(True)
###Unselect All Kanwils
    if event == "uall":
        for x in range (1, 13):
            window[str(x)].update(False)
###Run Search
    if event == "run":
	#get data from gui
        s=values['sentence']
        n=values['novs']
        c=values['qtf']
        q=all
	#change q accordingly
        match c:
            case "BSQ Layanan":
                q=bsq_l
            case "BSQ Digital":
                q=bsq_d
            case "CE Branch Experience":
                q=ce_be
            case "CE Branch Relationship":
                q=ce_br
	#select list of kanwils
        search=[]
        for x in range (1, 13):
            if values[str(x)] == True:
                search.append(x)
	#error handling
        if data.empty:
            sg.Popup('Tidak ada data yang sesuai kriteria!')
        elif search==[]:
            sg.Popup('Pilih minimal satu kanwil!')
        elif not s:
            sg.Popup('Masukkan teks untuk dicari!')
        else:
	    #filter out kanwil selection
            data["Kanwil"] = pd.to_numeric(data["Kanwil"], errors='coerce')
            data_proj = data.loc[data["Kanwil"].isin(search)].copy()
            if q != all: #convert q to numbers
                data_proj['Question Type'] = data_proj['Question Type'].str.replace('\D', '', regex=True)
                data_proj['Question Type'] = pd.to_numeric(data_proj['Question Type'])
                data_proj = data_proj[data_proj['Question Type'].isin(q)]
            data_proj = data_proj.reset_index(drop=True)
            t=get_most_similar_sentences(s,data_proj,n)
            window['results'].update(value=t, visible=True)
            if values['savef'] == True: #save search results if checked
                kws_l=','.join(map(str,search))
                fn="Verbatim Search '" + s + "' pada " + ntpath.basename(fta) + " KW " + kws_l + " - " + c + " (Top " + str(n) + ")"
                t.to_csv(r'{}.csv'.format(fn), sep=";", index=False)
    if event == "backf": #back to menu from finder
        window['header'].update("Verbatim Analyzer")
        window['mainmenu'].update(visible=True)
        window['findermenu'].update(visible=False)
        window['results'].update(value='')
window.Close()